Need to clean the notebook before the meeting

CSSG info session preparation

In [1]:
import numpy as np
import om10
import astropy.io.fits
import astropy
from astropy.io import fits

/Users/Jenny/anaconda/envs/python2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


1.txt contains Lens Name . . . u g r i z . . .

2.txt contains LensName Redshift . . . . . . .

3.txt contains Name	Obs. Date	Instrument	Slit	Width	Seeing	Exp. Time	z_d	z_s	sigma	S/N	Resolution

I need to merge all the textfiles. III_a and III_b is III table 4 and 5
IV is table 3

In [2]:
III_a = np.genfromtxt('../data/1.txt', dtype=str, usecols = (0, 4, 5, 6, 7, 8), invalid_raise=False, missing_values='xxx', usemask=False)
III_b = np.genfromtxt('../data/2.txt', dtype=str, usecols = (0, 1), invalid_raise=False, missing_values='xxx')
IV = np.genfromtxt('../data/3.txt', dtype=str, usecols = (0, 11), invalid_raise=False, missing_values='xxx')
name = np.array([])
redshift = np.array([])
veldisp = np.array([])
u = np.array([])
g = np.array([])
r = np.array([])
i = np.array([])
z = np.array([])
# O(n)
for (lensName, aIndex) in zip(III_a[:,0], range(len(III_a))):
    if lensName in III_b[:,0]:
        # assuming one occurence
        bIndex = np.argwhere(III_b[:,0]==lensName)[0][0]
        if lensName in IV[:,0]:
            VIndex = np.argwhere(IV[:,0]==lensName)[0][0]
            # only redshift has the problem
            if(IV[VIndex][1].isdigit()):
                name = np.append(name, lensName)
                redshift = np.append(redshift, float(III_b[bIndex][1]))
                veldisp = np.append(veldisp, float(IV[VIndex][1]))
                u = np.append(u, float(III_a[aIndex][1]))
                g = np.append(g, float(III_a[aIndex][2]))
                r = np.append(r, float(III_a[aIndex][3]))
                i = np.append(i, float(III_a[aIndex][4]))
                z = np.append(z, float(III_a[aIndex][5]))
from astropy.table import Table
table = Table([name, redshift, veldisp, u, g, r, i, z], names=['ID', 'ZLENS', 'VELDISP', 'MAGU', 'MAGG', 'MAGR', 'MAGI', 'MAGZ'])
np.savetxt('merged.txt', (name,redshift,veldisp,u,g,r,i,z), header="% name redshift veldisp u g r i z", fmt='%s') 

/Users/Jenny/anaconda/envs/python2/lib/python2.7/site-packages/numpy/lib/npyio.py:1772: ConversionWarning: Some errors were detected !
    Line #7 (got 1 columns instead of 2)
    Line #36 (got 1 columns instead of 2)
    Line #41 (got 1 columns instead of 2)
    Line #45 (got 1 columns instead of 2)
  warnings.warn(errmsg, ConversionWarning)
/Users/Jenny/anaconda/envs/python2/lib/python2.7/site-packages/numpy/lib/npyio.py:1772: ConversionWarning: Some errors were detected !
    Line #12 (got 11 columns instead of 2)
    Line #41 (got 11 columns instead of 2)
    Line #49 (got 11 columns instead of 2)
    Line #66 (got 11 columns instead of 2)
    Line #71 (got 11 columns instead of 2)
  warnings.warn(errmsg, ConversionWarning)


column : name redshift veldisp u g r i z

In [3]:
%%capture
#from astropy.table import Table
#table = Table.read('merged.txt', format='ascii')
#meta={'name': 'first table'}
db = om10.DB(catalog='/Users/Jenny/Desktop/om10fork/om10/data/qso_mock.fits')
db.sample = table
db.lenses = table

In [4]:
%%capture
import matplotlib
%matplotlib inline
matplotlib.rc('text', usetex=False)
gr = g-r
ri = r-i
iz = i-z
data = Table({'MAGI': i, 'ZLENS': z, 'GR': gr, 'RI': ri, 'IZ': iz}, names=['MAGI', 'ZLENS', 'GR', 'RI', 'IZ'])
axis_labels = {}
axis_labels['ZLENS'] = '$redshift$'
axis_labels['MAGI'] = '$i$'
axis_labels['GR'] = '$g-r$'
axis_labels['RI'] = '$r-i$'
axis_labels['IZ'] = '$i-z$'
db = om10.DB(catalog='/Users/Jenny/Desktop/om10fork/om10/data/qso_mock.fits')
db.select_random(Nlens=200,maglim=23.6)
# okay so speed did not change at all
db.paint(synthetic=True, target='lens')
db.gaussian_reweight(np.mean(z), np.std(z))
fig = om10.plot_sample(db.sample, color='blue', weight=True, parameters=('MAGI','ZLENS', 'GR', 'RI', 'IZ'))

UnboundLocalError: local variable 'ymin' referenced before assignment

In [5]:
from om10 import plotting
import corner
features, labels = plotting.extract_features(data, ('MAGI','ZLENS', 'GR', 'RI', 'IZ'))
corner.corner(features, labels=labels, color='red', smooth=1.0, fig=fig1)

NameError: name 'fig1' is not defined